In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
#import optuna
import seaborn as sns
from scipy.integrate import cumtrapz
import matplotlib.pyplot as plt
import time

data_dir = './data/'

Xtrain = pd.read_csv(data_dir + 'train_features.csv')
Xtest = pd.read_csv(data_dir + 'test_features.csv')
Ytrain = pd.read_csv(data_dir + 'train_labels.csv', index_col='id')
submission = pd.read_csv(data_dir + 'sample_submission.csv', index_col='id')
print(Ytrain.value_counts())

label  label_desc                                  
26     Non-Exercise                                    1518
8      Device on Table                                   97
28     Plank                                             55
60     Wall Squat                                        48
18     Jump Rope                                         47
                                                       ... 
53     Triceps Kickback (knee on bench) (right arm)      13
13     Dumbbell Row (knee on bench) (right arm)          12
12     Dumbbell Row (knee on bench) (left arm)           12
52     Triceps Kickback (knee on bench) (left arm)       12
0      Arm Band Adjustment                               12
Length: 61, dtype: int64


Imbalance 꽤 심한거 확인됨. 

In [3]:
feature_names = ['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']

grad_cols=[]
for col in feature_names:
    grad_cols.append(f"grad_{col}")

integ_cols = []
for col in feature_names:
    integ_cols.append(f"integ_{col}")
    
#position_cols = ['pos_x','pos_y','pos_z']
    
total_feature_names = feature_names + grad_cols + integ_cols #+ position_cols

- 가속도, 각속도 데이터이므로, 미분 적분을 구해서 가가속도, 각가속도, 속도, 각도 데이터를 만들어 Feature Engineering 할 예정.  
- UID 별로 600스텝마다 다른 데이터이므로 600스텝마다 잘라서 미분, 적분해야 함. 
- 속도를 한번 더 적분해서 만든 변위 데이터는 뺐을때 결과가 더 좋았음.  
    - 구분구적법 자체의 오차, 적분상수를 모른 채로 또 다시 적분을 시도해서 데이터가 엉망이 된다던가.. 했을 것 같음...

# EDA

## Gradient of acc, dps

In [4]:
for uid in tqdm(Xtrain['id'].unique()):
    temp = Xtrain.loc[Xtrain['id']==uid, feature_names]
    grad = np.gradient(temp, axis=0)
    Xtrain.loc[Xtrain['id']==uid, grad_cols] = grad
    
for uid in tqdm(Xtest['id'].unique()):
    temp = Xtest.loc[Xtest['id']==uid, feature_names]
    grad = np.gradient(temp, axis=0)
    Xtest.loc[Xtest['id']==uid, grad_cols] = grad
    
print(Xtrain.shape, Xtest.shape)
Xtrain.head()

100%|██████████| 782/782 [00:08<00:00, 93.44it/s]

(1875000, 14) (469200, 14)


,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,grad_acc_x,grad_acc_y,grad_acc_z,grad_gy_x,grad_gy_y,grad_gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,0.081610,-0.019602,-0.033997,0.894708,-8.590094,6.748896
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,0.049261,-0.007872,-0.052468,-1.512835,-6.786777,3.328242
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,0.002699,-0.015696,-0.016383,1.204943,-7.229370,-1.263398
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,-0.001861,0.003679,0.015430,3.951992,-6.891998,-1.470803
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,-0.001896,0.007851,-0.017338,3.676439,-5.326397,-0.916218


## Integration of acc, dps (speed, degree)

In [5]:
for uid in tqdm(Xtrain['id'].unique()):
    temp = Xtrain.loc[Xtrain['id']==uid, feature_names]
    integ = cumtrapz(temp, initial=0, axis=0)
    Xtrain.loc[Xtrain['id']==uid, integ_cols] = integ
    
for uid in tqdm(Xtest['id'].unique()):
    temp = Xtest.loc[Xtest['id']==uid, feature_names]
    integ = cumtrapz(temp, initial=0, axis=0)
    Xtest.loc[Xtest['id']==uid, integ_cols] = integ
    
print(Xtrain.shape, Xtest.shape)
Xtrain.head()

100%|██████████| 782/782 [00:08<00:00, 93.26it/s]

(1875000, 20) (469200, 20)


,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,grad_acc_x,grad_acc_y,grad_acc_z,grad_gy_x,grad_gy_y,grad_gy_z,integ_acc_x,integ_acc_y,integ_acc_z,integ_gy_x,integ_gy_y,integ_gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,0.081610,-0.019602,-0.033997,0.894708,-8.590094,6.748896,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,0.049261,-0.007872,-0.052468,-1.512835,-6.786777,3.328242,1.246892,-0.189173,-0.165445,-0.144254,-34.844056,-28.301664
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,0.002699,-0.015696,-0.016383,1.204943,-7.229370,-1.263398,2.543044,-0.386217,-0.383358,-1.801342,-76.474890,-53.275086
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,-0.001861,0.003679,0.015430,3.951992,-6.891998,-1.470803,3.841896,-0.598957,-0.617655,-2.253488,-125.335094,-79.511907
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,-0.001896,0.007851,-0.017338,3.676439,-5.326397,-0.916218,5.138887,-0.808019,-0.836521,1.246358,-181.087295,-107.219530


## Scale Data

In [6]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
scaler = StandardScaler()

Xtrain_scaled = pd.DataFrame(data=scaler.fit_transform(Xtrain[total_feature_names]), index=Xtrain.index, columns=total_feature_names)
Xtest_scaled = pd.DataFrame(data=scaler.transform(Xtest[total_feature_names]), index=Xtest.index, columns=total_feature_names)

print(Xtrain_scaled.shape, Xtest_scaled.shape)

(1875000, 18) (469200, 18)


## Reshape

In [7]:
Xtrain_scaled = np.array(Xtrain_scaled[total_feature_names]).reshape(-1, 600, len(total_feature_names)).astype('float32')
Xtest_scaled= np.array(Xtest_scaled[total_feature_names]).reshape(-1, 600, len(total_feature_names)).astype('float32')

Ytrain = Ytrain['label']

print(Xtrain_scaled.shape, Ytrain.shape, Xtest_scaled.shape, submission.shape)

(3125, 600, 18) (3125,) (782, 600, 18) (782, 61)


In [8]:
np.savez_compressed('data/jamm.npz', X_train=Xtrain_scaled, Y_train=Ytrain, X_test=Xtest_scaled)

# Modeling 

In [7]:
import keras
import random
import keras.backend as K
from keras.layers import *
from keras.models import Model
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold, KFold
import tensorflow as tf

# 데이터를 하나하나마다 다른 Rolling 과 다른 노이즈를 추가하여 오버샘플링 하는 용도의 함수
def aug_data(w, noise=True, roll_max=550, roll_min=50, noise_std=0.02):
    assert w.ndim == 3
    auged=[]

    for i in range(w.shape[0]):
        roll_amount = np.random.randint(roll_min, roll_max)
        data = np.roll(w[i:i+1], shift=roll_amount, axis=1)
        if noise:
            gaussian_noise = np.random.normal(0, noise_std, data.shape)
            data += gaussian_noise
        auged.append(data)
    
    auged = np.concatenate(auged)
    return auged


# 모델의 인풋 바로 다음에 랜덤한 값으로 Rolling 을 하는 커스텀 레이어. 
class Rolling(Layer):
    def __init__(self, roll_max=599, roll_min=0):
        super(Rolling, self).__init__()
        self.random_roll = random.randint(roll_min, roll_max)   
        
    #def build(self, input_shape):  # Create the state of the layer (weights)
    #    pass
    
    def call(self, inputs, training=None):# Defines the computation from inputs to outputs
        if training:
            return tf.roll(inputs, shift=self.random_roll, axis=1)
        else:
            return inputs
        
    def get_config(self):
        return {'random_roll': self.random_roll}


#GPU 메모리가 생각보다 많이 안필요한것 같아서, 메모리 60%로 제한. 
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
#config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

- 26번 Label이 Non-Excercise 이고 압도적으로 많으므로, 데이터상에서 26번항목이 아닌 것들만 한번 더 Rolling Augmentation을 하고 노이즈를 약간 추가하는 오버샘플링을 하여 train set 에 추가.  
- 모델이 생각보다 학습을 꽤 잘해서, 전체 데이터에 대해서 적용 후 train - valid set 을 나누면 train-valid 간 leak 이 생겨서 점수가 비정상적으로 높아질 수 있다고 판단하여, train - valid 를 StratifiedKFold 로 나눈 이후, train 셋에 대해서만 augment, oversampling 진행. 
- Dobby님이 올려주신 데이터 Rolling 아이디어를 커스텀 층으로 구현하여 데이터 입력 이후 매번 랜덤하게 Roll 될 수 있도록 구현. (Test 시에는 작동하지 않음)

In [8]:
# Convolution, Dense 레이어 여러번 쓰기 번거로우니까 만든 함수
def ConvBlock3(w, kernel_size, filter_size, activation):
    x_res = Conv1D(filter_size, kernel_size, kernel_initializer='he_uniform', padding='same')(w)
    x = BatchNormalization()(x_res)
    x = Activation(activation)(x)
    x = Conv1D(filter_size, kernel_size, kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv1D(filter_size, kernel_size, kernel_initializer='he_uniform', padding='same')(x)
    x = Add()([x, x_res])
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x

def DenseBNAct(w, dense_units, activation):
    x = Dense(dense_units, kernel_initializer='he_uniform')(w)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x



# CV 해서 train 데이터의 cross_val_predict 와 test 의 Averaged Prediction 을 리턴할 수 있도록 만듬. 
def cross_validate(build_fn, build_fn_params={}, X=Xtrain_scaled, Y=Ytrain, Xt=Xtest_scaled, batch_size=16, random_state=None,
                   num_folds=10, stratify=True, 
                   predict_train=True, predict_test=True, verbose=False, 
                   aug=False, aug_noise=True, aug_noise_std=0.03, num_augs=1, 
                   save=False, save_filename_prefix=None):
    if save:
        assert save_filename_prefix != None
        
    if stratify:
        kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    else:
        kf = KFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    
    
    if predict_train:
        train_pred = np.zeros(shape=(X.shape[0], 61))
    
    score = 0
    num_fold = 1
    if predict_test:
        model_list=[]
        test_pred = np.zeros(shape=submission.shape)
        
    for train_ind, valid_ind in tqdm(kf.split(X, Y)):
        x_train = X[train_ind]
        y_train = Y[train_ind]
        x_valid = X[valid_ind]
        y_valid = Y[valid_ind]
        
        if aug:
            aug_ind = y_train.loc[y_train!=26].index
            for _ in range(num_augs):
                aug_x = aug_data(X[aug_ind], noise=aug_noise, noise_std=aug_noise_std)
                aug_y = Y[aug_ind]
                x_train = np.concatenate([x_train, aug_x], axis=0)
                y_train = pd.concat((y_train, aug_y), axis=0)
                assert x_train.shape[0] == y_train.shape[0]

        model = build_fn(**build_fn_params)
        reduce_lr = ReduceLROnPlateau(mode='min', monitor='val_loss', verbose=verbose, factor=0.3, patience=25)
        es = EarlyStopping(mode='min', monitor='val_loss', restore_best_weights=True, verbose=verbose, patience=76)
        fit_history = model.fit(x_train, y_train, epochs=5000, shuffle=True, verbose=verbose, 
                                callbacks=[es, reduce_lr], batch_size=batch_size, validation_data=(x_valid, y_valid)
                               )
        score += min(fit_history.history['val_loss'])
        
        if verbose:
            print(f"Fold {num_fold} Score : {min(fit_history.history['val_loss'])}")
        if save:
            model.save(f"{save_filename_prefix} - Fold{num_fold} Model.h5")
        
        
        if predict_train:
            train_pred[valid_ind] = model.predict(x_valid)
        if predict_test:
            model_list.append(model)
            
        num_fold += 1

    print(f"Score : {score / num_folds}")
    return_list=[]
    if predict_train:
        train_pred = pd.DataFrame(data=train_pred, index=Y.index, columns=submission.columns)
        return_list.append(train_pred)
        
    if predict_test:
        for model in tqdm(model_list):
            test_pred += model.predict(Xt)
        
        return_list.append(pd.DataFrame(data=test_pred/num_folds, index=submission.index, columns=submission.columns))
        
    return_list.append(score / num_folds)
        
    return return_list

In [9]:
def build_fn(lr = 0.001):
    activation='elu'
    kernel_size=9
    
    
    model_in = Input(shape=Xtrain_scaled.shape[1:])
    x = Rolling(roll_max=599, roll_min=0)(model_in)
    x = SpatialDropout1D(0.1)(x)
    
    ##################################### FIX #####################################
    x = ConvBlock3(x, kernel_size=kernel_size, filter_size=128, activation=activation)
    x = MaxPooling1D(3)(x)
    x = SpatialDropout1D(0.1)(x)
    
    
    x = ConvBlock3(x, kernel_size=kernel_size, filter_size=128, activation=activation)
    x = GlobalAveragePooling1D()(x)
    ###############################################################################
    
    x = DenseBNAct(x, dense_units=64, activation=activation)
    x = Dropout(0.4)(x)
    
    
    model_out = Dense(units=61, activation='softmax')(x)
    model = Model(model_in, model_out)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Nadam(learning_rate=lr), metrics='accuracy')
    
    return model


build_fn().summary()

cv_preds = cross_validate(build_fn, verbose=True, random_state=2021, batch_size=16, 
                        stratify=True, num_folds=10, predict_train=True, predict_test=True, 
                        aug=True, aug_noise_std=0.03, num_augs=1,
                         save=True, save_filename_prefix='Try9 - FinalCV_1')

0it [00:00, ?it/s]

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 600, 18)]    0                                            
__________________________________________________________________________________________________
rolling (Rolling)               (None, 600, 18)      0           input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 600, 18)      0           rolling[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 600, 128)     20864       spatial_dropout1d[0][0]          
_______________________________________________________________________________________

1it [16:05, 965.73s/it]

Epoch 1/5000
267/267 [==============================] - 4s 16ms/step - loss: 3.0962 - accuracy: 0.3208 - val_loss: 3.0888 - val_accuracy: 0.2939
Epoch 2/5000
267/267 [==============================] - 4s 15ms/step - loss: 2.4813 - accuracy: 0.4016 - val_loss: 1.7691 - val_accuracy: 0.5719
Epoch 3/5000
267/267 [==============================] - 4s 15ms/step - loss: 2.1812 - accuracy: 0.4373 - val_loss: 1.4219 - val_accuracy: 0.6358
Epoch 4/5000
267/267 [==============================] - 4s 15ms/step - loss: 1.9925 - accuracy: 0.4617 - val_loss: 1.6805 - val_accuracy: 0.5527
Epoch 5/5000
267/267 [==============================] - 4s 15ms/step - loss: 1.9353 - accuracy: 0.4732 - val_loss: 1.2657 - val_accuracy: 0.6518
Epoch 6/5000
267/267 [==============================] - 4s 15ms/step - loss: 1.7781 - accuracy: 0.5016 - val_loss: 1.1850 - val_accuracy: 0.6486
Epoch 7/5000
267/267 [==============================] - 4s 15ms/step - loss: 1.6834 - accuracy: 0.5235 - val_loss: 1.3943 - val_ac

2it [32:36, 973.22s/it]

...

9it [2:00:27, 776.11s/it]

Epoch 1/5000
267/267 [==============================] - 5s 19ms/step - loss: 3.0514 - accuracy: 0.3197 - val_loss: 2.6551 - val_accuracy: 0.3974
Epoch 2/5000
267/267 [==============================] - 5s 18ms/step - loss: 2.4392 - accuracy: 0.3972 - val_loss: 1.6181 - val_accuracy: 0.5929
Epoch 3/5000
267/267 [==============================] - 5s 18ms/step - loss: 2.1608 - accuracy: 0.4378 - val_loss: 1.4057 - val_accuracy: 0.6282
Epoch 4/5000
267/267 [==============================] - 5s 17ms/step - loss: 1.9673 - accuracy: 0.4716 - val_loss: 1.2455 - val_accuracy: 0.6731
Epoch 5/5000
267/267 [==============================] - 5s 17ms/step - loss: 1.8525 - accuracy: 0.4937 - val_loss: 1.1712 - val_accuracy: 0.6667
Epoch 6/5000
267/267 [==============================] - 5s 17ms/step - loss: 1.7147 - accuracy: 0.5192 - val_loss: 1.1345 - val_accuracy: 0.6955
Epoch 7/5000
267/267 [==============================] - 5s 17ms/step - loss: 1.6425 - accuracy: 0.5305 - val_loss: 1.0298 - val_ac

10it [2:13:46, 802.62s/it]
 10%|████████▎                                                                          | 1/10 [00:00<00:01,  5.85it/s]

Score : 0.4934369385242462


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.55it/s]


In [10]:
cv_preds[1].to_csv('Try9FINAL-Score 0.49343693852,1AUG,0.03-NoAttention-ES76-InputDrop0.1,ConvDrop0.1,DenseDrop0.4.csv')